# MLP-Train Water with OpenMM on Colab

You can run this notebook in Google Colab: [![Open On Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sef43/mlp-train/blob/main/examples/water_openmm_colab.ipynb)

MLP-Train basic example that runs on Colab and uses OpenMM as the MD engine.

In [ ]:
# Installation on Colab
!pip install -q condacolab
import condacolab

condacolab.install_mambaforge()
# https://github.com/openmm/openmm-torch/issues/88
%env CONDA_OVERRIDE_CUDA=12.0
!mamba install -c conda-forge openmm-torch=1.1 pytorch=2.0 xtb cython rdkit

# Only have python 3.10 available on Colab so cannot install autodE from
# conda-forge as there are no py3.10 builds for autode=v1.1.*
!git clone --branch v1.1.3 https://github.com/duartegroup/autodE.git
!cd autodE && python setup.py install
!cd ..
!pip install ./autodE
!pip install git+https://github.com/rosswhitfield/ase.git
!pip install git+https://github.com/ACEsuit/mace.git
!pip install git+https://github.com/sef43/openmm-ml@mace
!pip install git+https://github.com/sef43/mlp-train

In [ ]:
# Get the example input
!wget https://raw.githubusercontent.com/sef43/mlp-train/main/examples/water.xyz

In [ ]:
import mlptrain as mlt

mlt.Config.n_cores = 1

system = mlt.System(mlt.Molecule('water.xyz'), box=None)

mace = mlt.potentials.MACE('water', system=system)

mace.al_train(
    method_name='xtb',
    temp=500,
    max_active_iters=2,
    max_active_time=50,
    n_configs_iter=3,
    md_program='OpenMM',
)

# Run some dynamics with the potential
trajectory = mlt.md_openmm.run_mlp_md_openmm(
    configuration=system.random_configuration(),
    mlp=mace,
    fs=2000,
    temp=300,
    dt=0.5,
    interval=100,
)

trajectory.save(filename='water_trajectory.xyz')